<a href="https://colab.research.google.com/github/AI4GoodE1/AI4GoodE1/blob/main/SMS_dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install datasets

In [45]:
#imports
import numpy as np
import pandas as pd

#Huggingface
from datasets import load_dataset

# Libraries needed for NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

The SMS dataset: **Cleaning**

In [46]:
dataset = load_dataset("sms_spam")

Reusing dataset sms_spam (C:\Users\shane\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|██████████| 1/1 [00:00<00:00, 432.49it/s]


In [47]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})

In [48]:
dataset = dataset.rename_column("sms", "Text")
dataset = dataset.rename_column("label", "Class")
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'Class'],
        num_rows: 5574
    })
})

In [49]:
dataset = dataset["train"]

In [50]:
dataset.to_csv('dataset/sms_dataset_raw.csv', index=False)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 26.56ba/s]


488667

In [51]:
df = pd.read_csv('dataset/sms_dataset_raw.csv')
df

,Text,Class
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5569,This is the 2nd time we have tried 2 contact u...,1
5570,Will ü b going to esplanade fr home?\n,0
5571,"Pity, * was in mood for that. So...any other s...",0
5572,The guy did some bitching but I acted like i'd...,0


In [52]:
# Check for NaN values in data
df1 = df[df.isna().any(axis=1)]
df1

,Text,Class


In [53]:
# Check for duplicates in data
print("Total: " + str(len(df)))
print("Unique: " + str(df["Text"].nunique()))

Total: 5574
Unique: 5171


In [54]:
#Remove Duplicates
print("Target",df["Text"].nunique())
print("Before",len(df))
#pd.set_option("max_rows", None)
df_unique = df.drop_duplicates(subset = "Text", keep='first')
print("After",len(df_unique))

Target 5171
Before 5574
After 5171


In [55]:
# Export cleaned dataset to csv
df_unique.to_csv('dataset/sms_cleaned_no_dups.csv', index = False)

SMS Dataset: **Pre-processing**

In [56]:
# Stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
# Lemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\shane\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [57]:
# Cleanup

# replace email address with 'emailaddress'
df['Text'] = df['Text'].str.replace(r'(<?)([A-Za-z0-9.]{1,30})@([A-Za-z0-9.]{1,30}).([a-z]{2,3})(>?)', 'emailaddress')

# replace urls with 'webaddress'
df['Text'] = df['Text'].str.replace(r"(https?:)(.?)(([^\s]+)|$)|(www.)(.?)(([^\s]+)|$)|(news.)(.*?)(([^\s]+)|$)", 'webaddress')

# replace 10 digit phone number with 'phone-number'
df['Text'] = df['Text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phone-number')

# replace normal number with 'number'
df['Text'] = df['Text'].str.replace(r'\d+(\.\d+)?', 'number')

# replace qzsoft strings
df['Text'] = df['Text'].str.replace(r'(qzsoft)(\S+)', ' qzsoft')

# replace broken MIME
df['Text'] = df['Text'].str.replace(r'([\S]{250,})', 'brokenmime')

# remove whitespace between terms with single space
df['Text'] = df['Text'].str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
df['Text'] = df['Text'].str.replace(r'^\s+|\s*?$', ' ')

# # replace html tags with 'format'
df['Text'] = df['Text'].str.replace(r"<(\"[^\"]*\"|'[^']*'|[^'\">])*>", ' formatting ')
df['Text'] = df['Text'].str.replace(r"(&nbsp;)", ' formatting ')

df['Text'] = df['Text'].str.replace(r"[_]{1,}", 'blankLines')

# remove punctuation
df['Text'] = df['Text'].str.replace(r'[^\w\d\s]', ' ')

# change words to lower case
df['Text'] = df['Text'].str.lower()

C:\Users\shane\AppData\Local\Temp\ipykernel_21612\260097670.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace(r'(<?)([A-Za-z0-9.]{1,30})@([A-Za-z0-9.]{1,30}).([a-z]{2,3})(>?)', 'emailaddress')
C:\Users\shane\AppData\Local\Temp\ipykernel_21612\260097670.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace(r"(https?:)(.?)(([^\s]+)|$)|(www.)(.?)(([^\s]+)|$)|(news.)(.*?)(([^\s]+)|$)", 'webaddress')
C:\Users\shane\AppData\Local\Temp\ipykernel_21612\260097670.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phone-number')
C:\Users\shane\AppData\Local\Temp\ipykernel_21612\260097670.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['T

In [58]:
# Applying English Stopwords
df['Text'] = df['Text'].apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
# Applying Wordnet Lemmatizer
df['Text'] = df['Text'].apply(lambda x: ' '.join(lemmatizer.lemmatize(term) for term in x.split()))

In [59]:
# Random sampling of data to investigate dataset
for i in range(0,10):
  v = df.sample()
  # print(v.iloc[0])
  print(v.iloc[0]["Text"])
  print(v.iloc[0]["Class"])
  print("-----------------------------------------------------------\n")

ow u dey paid number numberthousad told u would call
0
-----------------------------------------------------------

want explicit sex number sec ring number cost numberp min gsex pobox number wcnumbern numberxx
1
-----------------------------------------------------------

k yan jiu liao sat go number bugis vill one frm number number den hop parco number nb sun go cine frm number number den hop orc mrt number hip hop number
0
-----------------------------------------------------------

dear hero leaving qatar tonite apt opportunity pls keep touch lt email gt kerala
0
-----------------------------------------------------------

probs hon u doinat mo
0
-----------------------------------------------------------

sexy single waiting text age followed gender wither f e g numberf gay men text age followed g e g numberg
1
-----------------------------------------------------------

remains bro amongst bros
0
-----------------------------------------------------------

leave around four ok
0


In [60]:
#Export
# Export cleaned dataset to csv
df.to_csv('dataset/fraud_sms_preprocessed.csv', index = False)